# Import Dependencies

In [35]:
import pandas as pd
import numpy as np

from keras.models import load_model
import pickle

pd.set_option('display.max_columns', None)

# Data Acquisition

In [36]:
test = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\application_test.csv")
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married

# Data Cleaning

In [37]:
test.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 121 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    SK_ID_CURR                    48744 non-null  int64  
 1    NAME_CONTRACT_TYPE            48744 non-null  object 
 2    CODE_GENDER                   48744 non-null  object 
 3    FLAG_OWN_CAR                  48744 non-null  object 
 4    FLAG_OWN_REALTY               48744 non-null  object 
 5    CNT_CHILDREN                  48744 non-null  int64  
 6    AMT_INCOME_TOTAL              48744 non-null  float64
 7    AMT_CREDIT                    48744 non-null  float64
 8    AMT_ANNUITY                   48720 non-null  float64
 9    AMT_GOODS_PRICE               48744 non-null  float64
 10   NAME_TYPE_SUITE               47833 non-null  object 
 11   NAME_INCOME_TYPE              48744 non-null  object 
 12   NAME_EDUCATION_TYPE           48744 non-null

In [38]:
test.drop(columns=['FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL'], inplace=True)

In [39]:
test['AGE'] = test['DAYS_BIRTH']/-365
test['TENURE'] = test['DAYS_EMPLOYED']/-365
test['DAYS_REGISTRATION'] = test['DAYS_REGISTRATION']/-365
test['DAYS_ID_PUBLISH'] = test['DAYS_ID_PUBLISH']/-365
test['DAYS_LAST_PHONE_CHANGE'] = test['DAYS_LAST_PHONE_CHANGE']/-365

test['TENURE'] = np.where((test['TENURE']<0), 0, test['TENURE'])

test.drop(columns=['DAYS_BIRTH', 'DAYS_EMPLOYED'], inplace=True)

In [40]:
def fill_missing_external(df, column_name, pred_list):
    missing_indices = df[df[column_name].isnull()].index

    for i, index in enumerate(missing_indices):
        if i < len(pred_list):
            df.at[index, column_name] = pred_list[i]
        else:
            break

In [41]:
ext1_knn = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\ext1_knn.pickle", "rb"))
ext2_knn = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\ext2_knn.pickle", "rb"))
ext3_knn = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\ext3_knn.pickle", "rb"))

In [42]:
ext2_features = test[['REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','REGION_POPULATION_RELATIVE', 'EXT_SOURCE_2']]
ext2_features_nn = ext2_features.dropna()

ext2_null = ext2_features[ext2_features['EXT_SOURCE_2'].isna()].iloc[:, :3]
ext_2_pred = ext2_knn.predict(ext2_null)

In [43]:
fill_missing_external(test, 'EXT_SOURCE_2', ext_2_pred)

In [44]:
ext1_features = test[['AGE', 'EXT_SOURCE_2', 'EXT_SOURCE_1']]
ext1_features_nn = ext1_features.dropna()

ext1_null = ext1_features[ext1_features['EXT_SOURCE_1'].isna()].iloc[:, :2]
ext_1_pred = ext1_knn.predict(ext1_null)

In [45]:
fill_missing_external(test, 'EXT_SOURCE_1', ext_1_pred)

In [46]:
ext3_features = test[['AGE', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_3']]
ext3_features_nn = ext3_features.dropna()

ext3_null = ext3_features[ext3_features['EXT_SOURCE_3'].isna()].iloc[:, :4]
ext_3_pred = ext3_knn.predict(ext3_null)

In [47]:
fill_missing_external(test, 'EXT_SOURCE_3', ext_3_pred)

In [48]:
test['HOUR_APPR_PROCESS_START'] = test['HOUR_APPR_PROCESS_START'].astype('object')

In [49]:
test['OWN_CAR_AGE'] = np.where(((test['FLAG_OWN_CAR']=='N')&(test['OWN_CAR_AGE'].isna())), 0, test['OWN_CAR_AGE'])

test['OWN_CAR_AGE'] = np.where(((test['FLAG_OWN_CAR']=='Y')&(test['OWN_CAR_AGE'].isna())), 10, test['OWN_CAR_AGE'])

In [50]:
test['log_INCOME_TOTAL'] = np.log(test['AMT_INCOME_TOTAL'])

In [51]:
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Pensioner')&(test['ORGANIZATION_TYPE']=='XNA')), "XNA", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Pensioner')&(test['ORGANIZATION_TYPE']=='Trade: type 7')), "Sales staff", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Unemployed')&(test['ORGANIZATION_TYPE']=='XNA')), "XNA", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Working')), "Laborers", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Commercial associate')), "Laborers", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='State servant')), "Core staff", test['OCCUPATION_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='State servant')), "Core staff", test['OCCUPATION_TYPE'])
test['NAME_INCOME_TYPE'] = np.where((test['NAME_INCOME_TYPE']=='Student'), "Working", test['NAME_INCOME_TYPE'])
test['OCCUPATION_TYPE'] = np.where(((test['OCCUPATION_TYPE'].isna())&(test['NAME_INCOME_TYPE']=='Working')), "Other", test['OCCUPATION_TYPE'])

In [52]:
test.drop(columns="FLAG_DOCUMENT_12", inplace=True)

In [53]:
test['AMT_REQ_CREDIT_BUREAU_HOUR'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_HOUR'])
test['AMT_REQ_CREDIT_BUREAU_DAY'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_DAY'])
test['AMT_REQ_CREDIT_BUREAU_WEEK'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_WEEK'])
test['AMT_REQ_CREDIT_BUREAU_MON'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_MON'])
test['AMT_REQ_CREDIT_BUREAU_QRT'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_QRT'])
test['AMT_REQ_CREDIT_BUREAU_YEAR'] = np.where(((test['TENURE']==0)&(test['ORGANIZATION_TYPE']=='XNA')), 0, test['AMT_REQ_CREDIT_BUREAU_YEAR'])

In [54]:
test['NAME_HOUSING_TYPE'] = np.where(((test['HOUSETYPE_MODE'].isna())&(test['NAME_HOUSING_TYPE']=='House / apartment')), 'House', test['NAME_HOUSING_TYPE'])
test['HOUSETYPE_MODE'] = np.where((test['HOUSETYPE_MODE'].isna()), 'house', test['HOUSETYPE_MODE'])

test['FONDKAPREMONT_MODE'] = np.where((test['HOUSETYPE_MODE']=='house'), 'XNA', test['FONDKAPREMONT_MODE'])
test['WALLSMATERIAL_MODE'] = np.where((test['HOUSETYPE_MODE']=='house'), 'XNA', test['WALLSMATERIAL_MODE'])
test['EMERGENCYSTATE_MODE'] = np.where((test['HOUSETYPE_MODE']=='house'), 'XNA', test['EMERGENCYSTATE_MODE'])

test['FONDKAPREMONT_MODE'] = np.where(((test['FONDKAPREMONT_MODE'].isna())&(test['HOUSETYPE_MODE']!='house')), 'not specified', test['FONDKAPREMONT_MODE'])
test['WALLSMATERIAL_MODE'] = np.where(((test['WALLSMATERIAL_MODE'].isna())&(test['HOUSETYPE_MODE']=='specific housing')), 'Stone, brick', test['WALLSMATERIAL_MODE'])
test['WALLSMATERIAL_MODE'] = np.where(((test['WALLSMATERIAL_MODE'].isna())&(test['HOUSETYPE_MODE']=='terraced house')), 'Stone, brick', test['WALLSMATERIAL_MODE'])

In [55]:
test['CNT_FAM_MEMBERS'] = test['CNT_FAM_MEMBERS'].fillna(1.0)
test['NAME_TYPE_SUITE'] = test['NAME_TYPE_SUITE'].fillna('Unknown')
test['AMT_GOODS_PRICE'] = np.where((test['AMT_GOODS_PRICE'].isna()), test['AMT_CREDIT'], test['AMT_GOODS_PRICE'])
test['WALLSMATERIAL_MODE'] = test['WALLSMATERIAL_MODE'].fillna('Unknown')
test['AMT_ANNUITY'] = test['AMT_ANNUITY'].fillna(27652.5) #median of the amount annuity when the credit amount is 539,100
test.fillna(-1, inplace=True)

In [56]:
test.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 118 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    SK_ID_CURR                    48744 non-null  int64  
 1    NAME_CONTRACT_TYPE            48744 non-null  object 
 2    CODE_GENDER                   48744 non-null  object 
 3    FLAG_OWN_CAR                  48744 non-null  object 
 4    FLAG_OWN_REALTY               48744 non-null  object 
 5    CNT_CHILDREN                  48744 non-null  int64  
 6    AMT_INCOME_TOTAL              48744 non-null  float64
 7    AMT_CREDIT                    48744 non-null  float64
 8    AMT_ANNUITY                   48744 non-null  float64
 9    AMT_GOODS_PRICE               48744 non-null  float64
 10   NAME_TYPE_SUITE               48744 non-null  object 
 11   NAME_INCOME_TYPE              48744 non-null  object 
 12   NAME_EDUCATION_TYPE           48744 non-null

# Merge Datasets

In [57]:
bureau = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_bureau.csv")
credit = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_credit.csv")
instal = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_installment.csv")
cash = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_POS.csv")

In [58]:
final_test = test.merge(bureau, on='SK_ID_CURR', how='left')
final_test = final_test.merge(credit, on='SK_ID_CURR', how='left')
final_test = final_test.merge(instal, on='SK_ID_CURR', how='left')
final_test = final_test.merge(cash, on='SK_ID_CURR', how='left')
final_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE,TENURE,log_INCOME_TOTAL,BALANCE_FRESH,LAST_STATUS,CREDIT_COUNT,APPLY_CRED_MAX,APPLY_CRED_MIN,CRED_UPDATE,CRED_DEBT_MED,OVERDUE_CRED_SUM,CRED_SUM_MED,CRED_FRESH,CRED_OLD,MED_CRED_LIMIT,MED_CURRENT_DRAW,MED_AMOUNT_PAID,MED_AMOUNT_RECIEVE,MED_AMOUNT_RECIEVED,MED_TOTAL_RECIEVEABLE,MAX_DRAWINGS,MED_MINMAX_INST,COUNT_INSTALL,INST_PAY_RECENT,MED_PAID_INSTALL,INST_PAID,MED_INSTALL_AMT,MED_INSTALL_AMT_PAID,DIFF_INST_PAID_DAYS,DIFF_INST_PAID_AMT,NUM_CASH_LOAN,FRESH_CASH_BAL,OLD_CASH_BAL,AVG_CASH_INST,AVG_CASH_INST_LEFT,AVG_CASH_INST_PAID
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,14.164384,2.224658,0.0,1,0,0,Laborers,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,-1.0000,-1.0000,-1.00,0.1379,0.1250,-1.0000,-1.0000,-1.0000,0.0505,-1.0000,-1.0000,0.0672,0.0612,0.9732,-1.0000,-1.0000,-1.0000,0.1379,0.1250,-1.0000,-1.0000,-1.0000,0.0526,-1.0000,-1.0000,0.0666,0.0590,0.9732,-1.0000,-1.0000,-1.00,0.1379,0.1250,-1.0000,-1.0000,-1.0000,0.0514,-1.0000,-1.0000,not specified,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,4.767123,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,52.715068,6.380822,11.813030,-18.0,0,32.0,-49.0,-559.0,-6.0,113166.000,0.0,81972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,2916.0,3.0,2916.0,3980.9250,3980.9250,0.0,0.000,9.0,-53.0,-96.0,4.0,1.0,3.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House,0.035792,24.980822,4.446575,0.0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.00,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000

In [59]:
final_test.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 151 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    SK_ID_CURR                    48744 non-null  int64  
 1    NAME_CONTRACT_TYPE            48744 non-null  object 
 2    CODE_GENDER                   48744 non-null  object 
 3    FLAG_OWN_CAR                  48744 non-null  object 
 4    FLAG_OWN_REALTY               48744 non-null  object 
 5    CNT_CHILDREN                  48744 non-null  int64  
 6    AMT_INCOME_TOTAL              48744 non-null  float64
 7    AMT_CREDIT                    48744 non-null  float64
 8    AMT_ANNUITY                   48744 non-null  float64
 9    AMT_GOODS_PRICE               48744 non-null  float64
 10   NAME_TYPE_SUITE               48744 non-null  object 
 11   NAME_INCOME_TYPE              48744 non-null  object 
 12   NAME_EDUCATION_TYPE           48744 non-null

In [60]:
final_test['LAST_STATUS'] = final_test['LAST_STATUS'].fillna('Unknown')
final_test['LAST_STATUS'] = final_test['LAST_STATUS'].astype('object')
final_test.fillna(0, inplace=True)

In [61]:
final_test.isna().sum().sum()

0

# Data Preparation

In [62]:
final_test['FLAG_OWN_CAR'] = final_test['FLAG_OWN_CAR'].map(lambda x: 1 if x=='Y' else 0)
final_test['FLAG_OWN_REALTY'] = final_test['FLAG_OWN_REALTY'].map(lambda x: 1 if x=='Y' else 0)

In [63]:
final_test['NAME_HOUSING_TYPE'] = final_test['NAME_HOUSING_TYPE'].map(lambda x: 'Apartment' if x=='House / apartment' else x)

In [64]:
lower_limit = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\lower_limit.pickle", "rb"))
upper_limit = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\upper_limit.pickle", "rb"))
lower_limit2 = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\lower_limit2.pickle", "rb"))
upper_limit2 = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\upper_limit2.pickle", "rb"))

In [65]:
final_test['CRED_INCOME_RATIO'] = round(final_test['AMT_CREDIT']/final_test['AMT_INCOME_TOTAL']*100, 4)
final_test['GOODS_ANNUITY_RATIO'] = round(final_test['AMT_GOODS_PRICE']/final_test['AMT_ANNUITY']*100, 4)
final_test['ANNUITY_INCOME_RATIO'] = round(final_test['AMT_ANNUITY']/final_test['AMT_INCOME_TOTAL']*100, 4)
final_test['CREDIT_USAGE'] = round(final_test['AMT_GOODS_PRICE']/final_test['AMT_CREDIT']*100, 4)

final_test['INCOME_LEVEL'] = final_test['log_INCOME_TOTAL'].map(lambda x: 'High' if x >= upper_limit2 else 'Above Avg' if x>= upper_limit else 'Below Avg' if x<=lower_limit else 'Low')

final_test['TENURE_AGE_RATIO'] = round((final_test['TENURE']/final_test['AGE']), 4)
final_test['OWN_CAR_RATIO'] = round((final_test['OWN_CAR_AGE']/final_test['AGE']), 4)

final_test['ADDRESS_REGION_FLAG'] = final_test['REG_REGION_NOT_LIVE_REGION'] + final_test['REG_REGION_NOT_WORK_REGION'] + final_test['LIVE_REGION_NOT_WORK_REGION']
final_test['ADDRESS_REGION_FLAG'] = final_test['ADDRESS_REGION_FLAG'].map(lambda x: 1 if x > 0 else x)
final_test['ADDRESS_CITY_FLAG'] = final_test['REG_CITY_NOT_LIVE_CITY'] + final_test['REG_CITY_NOT_WORK_CITY'] + final_test['LIVE_CITY_NOT_WORK_CITY']
final_test['ADDRESS_CITY_FLAG'] = final_test['ADDRESS_CITY_FLAG'].map(lambda x: 1 if x > 0 else x)

final_test['LEFT_INCOME'] = final_test['AMT_INCOME_TOTAL'] - final_test['AMT_ANNUITY']

final_test['MONTHLY_INCOME'] = final_test['AMT_INCOME_TOTAL']/12
final_test['MONTHLY_INSTALLMENT'] = final_test['AMT_ANNUITY']/12

final_test['DTI'] = final_test['MONTHLY_INSTALLMENT']/final_test['MONTHLY_INCOME']

final_test.drop(columns=['AMT_INCOME_TOTAL', 'REG_REGION_NOT_LIVE_REGION','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_WORK_REGION','REG_CITY_NOT_WORK_CITY','LIVE_REGION_NOT_WORK_REGION','LIVE_CITY_NOT_WORK_CITY'], inplace=True)

In [66]:
numeric_features = final_test[final_test.drop(columns=['SK_ID_CURR']).select_dtypes(exclude='object').columns]

In [67]:
scaler = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\scaler.pickle", "rb"))

In [68]:
test_scaled = pd.DataFrame(scaler.transform(numeric_features), columns=numeric_features.columns)

In [69]:
test_scaled.shape

(48744, 139)

In [70]:
encode_list = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\encode_list.pickle", "rb"))

cat_features = final_test[encode_list]

onehot = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\onehot.pickle", "rb"))

test_encoded = onehot.transform(cat_features)

In [71]:
test_encoded.shape

(48744, 176)

In [72]:
test_final = test_scaled.join(test_encoded)

In [73]:
test_final.shape

(48744, 315)

In [74]:
test_final

,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE,TENURE,log_INCOME_TOTAL,BALANCE_FRESH,CREDIT_COUNT,APPLY_CRED_MAX,APPLY_CRED_MIN,CRED_UPDATE,CRED_DEBT_MED,OVERDUE_CRED_SUM,CRED_SUM_MED,CRED_FRESH,CRED_OLD,MED_CRED_LIMIT,MED_CURRENT_DRAW,MED_AMOUNT_PAID,MED_AMOUNT_RECIEVE,MED_AMOUNT_RECIEVED,MED_TOTAL_RECIEVEABLE,MAX_DRAWINGS,MED_MINMAX_INST,COUNT_INSTALL,INST_PAY_RECENT,MED_PAID_INSTALL,INST_PAID,MED_INSTALL_AMT,MED_INSTALL_AMT_PAID,DIFF_INST_PAID_DAYS,DIFF_INST_PAID_AMT,NUM_CASH_LOAN,FRESH_CASH_BAL,OLD_CASH_BAL,AVG_CASH_INST,AVG_CASH_INST_LEFT,AVG_CASH_INST_PAID,CRED_INCOME_RATIO,GOODS_ANNUITY_RATIO,ANNUITY_INCOME_RATIO,CREDIT_USAGE,TENURE_AGE_RATIO,OWN_CAR_RATIO,ADDRESS_REGION_FLAG,ADDRESS_CITY_FLAG,LEFT_INCOME,MONTHLY_INCOME,MONTHLY_INSTALLMENT,DTI,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_F,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Unaccompanied,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unknown,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Group of people,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House,NAME_HOUSING_TYPE_Apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Office apartment,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_XNA,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Waiters/barmen staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Private service staff,O

# Model Prediction

In [78]:
rfc_model = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\rfc_model.pickle", "rb"))
xgb_model = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\xgb_model.pickle", "rb"))
logi_model = pickle.load(open(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\logi_model.pickle", "rb"))


rfc_pred = rfc_model.predict_proba(test_final)
xgb_pred = xgb_model.predict_proba(test_final)
logi_pred = logi_model.predict_proba(test_final)

In [100]:
weighted_pred = rfc_pred * 0.3 + xgb_pred * 0.4 + logi_pred * 0.3
weighted_pred

array([[0.54837474, 0.45162527],
       [0.24973745, 0.75026255],
       [0.5827904 , 0.4172096 ],
       ...,
       [0.75697574, 0.24302426],
       [0.63991072, 0.36008927],
       [0.27675598, 0.72324402]])

In [101]:
result_df = pd.DataFrame(weighted_pred)
result = result_df[1]
result

0        0.451625
1        0.750263
2        0.417210
3        0.330068
4        0.728093
           ...   
48739    0.505061
48740    0.437169
48741    0.243024
48742    0.360089
48743    0.723244
Name: 1, Length: 48744, dtype: float64

In [102]:
submission = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Datasets\home-credit-default-risk\sample_submission.csv")

submission.drop(columns='TARGET', inplace=True)
submission = submission.join(result)
submission.columns = ['SK_ID_CURR', 'TARGET']

submission

,SK_ID_CURR,TARGET
0,100001,0.451625
1,100005,0.750263
2,100013,0.417210
3,100028,0.330068
4,100038,0.728093
...,...,...
48739,456221,0.505061
48740,456222,0.437169
48741,456223,0.243024
48742,456224,0.360089


In [103]:
submission.to_csv("submission.csv", index=False)

# Neural Networks

In [138]:
ann_model = load_model(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\neural_model.keras")

In [139]:
pred = ann_model.predict(test_final)

1524/1524 [==============================] - 1s 582us/step


In [144]:
result_df = pd.DataFrame(pred)
result_df

,0
0,0.453365
1,0.832376
2,0.368828
3,0.324226
4,0.826221
...,...
48739,0.460794
48740,0.461890
48741,0.200581
48742,0.409714


In [145]:
submission = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Datasets\home-credit-default-risk\sample_submission.csv")

submission.drop(columns='TARGET', inplace=True)
submission = submission.join(result_df)
submission.columns = ['SK_ID_CURR', 'TARGET']

submission

,SK_ID_CURR,TARGET
0,100001,0.453365
1,100005,0.832376
2,100013,0.368828
3,100028,0.324226
4,100038,0.826221
...,...,...
48739,456221,0.460794
48740,456222,0.461890
48741,456223,0.200581
48742,456224,0.409714


In [146]:
submission.to_csv("ann_sub.csv", index=False)